In [1]:
import torch
import torch.nn as nn
import cv2
import csv
import json
import torchvision.models as models
import numpy as np
import pandas as pd
import statistics
import threading
# from object_detection import ObjectDetection
import math
import subprocess
from tqdm import tqdm
from torchvision import transforms
# import maths

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True)
model.eval();

class Image_Classifier(nn.Module):
    def init(self):
        super().init()
        self.model = nn.Sequential(
             Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1)),
             ReLU(),
             Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1)),
             ReLU(),
             MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
             Flatten(start_dim=1, end_dim=-1),
             Dropout(p=0.25, inplace=False),
             Linear(in_features=6272, out_features=132, bias=True),
             ReLU(),
             Dropout(p=0.5, inplace=False),
             Linear(in_features=132, out_features=11, bias=True),
        )

    def forward(self, x):
        return self.model(x)

color_classifier = torch.load("color.pt")
color_classifier = color_classifier.cuda()
color_classifier.eval()

body_classifier = torch.load('model.pt')

Using cache found in C:\Users\ahmed/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-4-18 Python-3.9.13 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)



requirements: C:\Users\ahmed\.cache\torch\hub\requirements.txt not found, check failed.


Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 


In [14]:
"""
    SORT: A Simple, Online and Realtime Tracker
    Copyright (C) 2016-2020 Alex Bewley alex@bewley.ai

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""
from __future__ import print_function

import os
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import io

import glob
import time
import argparse
from filterpy.kalman import KalmanFilter

np.random.seed(0)


def linear_assignment(cost_matrix):
  try:
    import lap
    _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
    return np.array([[y[i],i] for i in x if i >= 0]) #
  except ImportError:
    from scipy.optimize import linear_sum_assignment
    x, y = linear_sum_assignment(cost_matrix)
    return np.array(list(zip(x, y)))


def iou_batch(bb_test, bb_gt):
  """
  From SORT: Computes IOU between two bboxes in the form [x1,y1,x2,y2]
  """
  bb_gt = np.expand_dims(bb_gt, 0)
  bb_test = np.expand_dims(bb_test, 1)
  
  xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
  yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
  xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
  yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
  w = np.maximum(0., xx2 - xx1)
  h = np.maximum(0., yy2 - yy1)
  wh = w * h
  o = wh / ((bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])                                      
    + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1]) - wh)                                              
  return(o)  


def convert_bbox_to_z(bbox):
  """
  Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
    [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
    the aspect ratio
  """
  w = bbox[2] - bbox[0]
  h = bbox[3] - bbox[1]
  x = bbox[0] + w/2.
  y = bbox[1] + h/2.
  s = w * h    #scale is just area
  r = w / float(h)
  return np.array([x, y, s, r]).reshape((4, 1))


def convert_x_to_bbox(x,score=None):
  """
  Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
    [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
  """
  w = np.sqrt(x[2] * x[3])
  h = x[2] / w
  if(score==None):
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
  else:
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))


class KalmanBoxTracker(object):
  """
  This class represents the internal state of individual tracked objects observed as bbox.
  """
  count = 0
  def __init__(self,bbox):
    """
    Initialises a tracker using initial bounding box.
    """
    #define constant velocity model
    self.kf = KalmanFilter(dim_x=7, dim_z=4) 
    self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
    self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])

    self.kf.R[2:,2:] *= 10.
    self.kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
    self.kf.P *= 10.
    self.kf.Q[-1,-1] *= 0.01
    self.kf.Q[4:,4:] *= 0.01

    self.kf.x[:4] = convert_bbox_to_z(bbox)
    self.time_since_update = 0
    self.id = KalmanBoxTracker.count
    KalmanBoxTracker.count += 1
    self.history = []
    self.hits = 0
    self.hit_streak = 0
    self.age = 0

  def update(self,bbox):
    """
    Updates the state vector with observed bbox.
    """
    self.time_since_update = 0
    self.history = []
    self.hits += 1
    self.hit_streak += 1
    self.kf.update(convert_bbox_to_z(bbox))

  def predict(self):
    """
    Advances the state vector and returns the predicted bounding box estimate.
    """
    if((self.kf.x[6]+self.kf.x[2])<=0):
      self.kf.x[6] *= 0.0
    self.kf.predict()
    self.age += 1
    if(self.time_since_update>0):
      self.hit_streak = 0
    self.time_since_update += 1
    self.history.append(convert_x_to_bbox(self.kf.x))
    return self.history[-1]

  def get_state(self):
    """
    Returns the current bounding box estimate.
    """
    return convert_x_to_bbox(self.kf.x)


def associate_detections_to_trackers(detections,trackers,iou_threshold = 0.3):
  """
  Assigns detections to tracked object (both represented as bounding boxes)

  Returns 3 lists of matches, unmatched_detections and unmatched_trackers
  """
  if(len(trackers)==0):
    return np.empty((0,2),dtype=int), np.arange(len(detections)), np.empty((0,5),dtype=int)

  iou_matrix = iou_batch(detections, trackers)

  if min(iou_matrix.shape) > 0:
    a = (iou_matrix > iou_threshold).astype(np.int32)
    if a.sum(1).max() == 1 and a.sum(0).max() == 1:
        matched_indices = np.stack(np.where(a), axis=1)
    else:
      matched_indices = linear_assignment(-iou_matrix)
  else:
    matched_indices = np.empty(shape=(0,2))

  unmatched_detections = []
  for d, det in enumerate(detections):
    if(d not in matched_indices[:,0]):
      unmatched_detections.append(d)
  unmatched_trackers = []
  for t, trk in enumerate(trackers):
    if(t not in matched_indices[:,1]):
      unmatched_trackers.append(t)

  #filter out matched with low IOU
  matches = []
  for m in matched_indices:
    if(iou_matrix[m[0], m[1]]<iou_threshold):
      unmatched_detections.append(m[0])
      unmatched_trackers.append(m[1])
    else:
      matches.append(m.reshape(1,2))
  if(len(matches)==0):
    matches = np.empty((0,2),dtype=int)
  else:
    matches = np.concatenate(matches,axis=0)

  return matches, np.array(unmatched_detections), np.array(unmatched_trackers)


class Sort(object):
  def __init__(self, max_age=1, min_hits=3, iou_threshold=0.3):
    """
    Sets key parameters for SORT
    """
    self.max_age = max_age
    self.min_hits = min_hits
    self.iou_threshold = iou_threshold
    self.trackers = []
    self.frame_count = 0

  def update(self, dets=np.empty((0, 5))):
    """
    Params:
      dets - a numpy array of detections in the format [[x1,y1,x2,y2,score],[x1,y1,x2,y2,score],...]
    Requires: this method must be called once for each frame even with empty detections (use np.empty((0, 5)) for frames without detections).
    Returns the a similar array, where the last column is the object ID.

    NOTE: The number of objects returned may differ from the number of detections provided.
    """
    self.frame_count += 1
    # get predicted locations from existing trackers.
    trks = np.zeros((len(self.trackers), 5))
    to_del = []
    ret = []
    for t, trk in enumerate(trks):
      pos = self.trackers[t].predict()[0]
      trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]
      if np.any(np.isnan(pos)):
        to_del.append(t)
    trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
    for t in reversed(to_del):
      self.trackers.pop(t)
    matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets,trks, self.iou_threshold)

    # update matched trackers with assigned detections
    for m in matched:
      self.trackers[m[1]].update(dets[m[0], :])

    # create and initialise new trackers for unmatched detections
    for i in unmatched_dets:
        trk = KalmanBoxTracker(dets[i,:])
        self.trackers.append(trk)
    i = len(self.trackers)
    for trk in reversed(self.trackers):
        d = trk.get_state()[0]
        if (trk.time_since_update < 1) and (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits):
          ret.append(np.concatenate((d,[trk.id+1])).reshape(1,-1)) # +1 as MOT benchmark requires positive
        i -= 1
        # remove dead tracklet
        if(trk.time_since_update > self.max_age):
          self.trackers.pop(i)
    if(len(ret)>0):
      return np.concatenate(ret)
    return np.empty((0,5))

def parse_args():
    """Parse input arguments."""
    parser = argparse.ArgumentParser(description='SORT demo')
    parser.add_argument('--display', dest='display', help='Display online tracker output (slow) [False]',action='store_true')
    parser.add_argument("--seq_path", help="Path to detections.", type=str, default='data')
    parser.add_argument("--phase", help="Subdirectory in seq_path.", type=str, default='train')
    parser.add_argument("--max_age", 
                        help="Maximum number of frames to keep alive a track without associated detections.", 
                        type=int, default=1)
    parser.add_argument("--min_hits", 
                        help="Minimum number of associated detections before track is initialised.", 
                        type=int, default=3)
    parser.add_argument("--iou_threshold", help="Minimum IOU for match.", type=float, default=0.3)
    args = parser.parse_args()
    return args

# if __name__ == '__main__':
#     video_path = 'traffic.mp4'
#     output_path = 'track_test1.mp4'
#     cap = cv2.VideoCapture(video_path)

#     fps = cap.get(cv2.CAP_PROP_FPS)
#     frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#     frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

#     ffmpeg_cmd = f"ffmpeg -y -f rawvideo -pix_fmt bgr24 -s {frame_width}x{frame_height} -r {fps} -i - -c:v libx264 -preset fast -crf 30 -pix_fmt nv12 -an -vcodec libx264 {output_path}"

#     output_file = subprocess.Popen(ffmpeg_cmd.split(' '), stdin=subprocess.PIPE)

#     mot_tracker = Sort(max_age=30, min_hits=60) 

#     object_dict = {}

#     frame_cut = 0
#     frame_count = 0
#     clf_state = False
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             print('Video processing completed')
#             break

#         frame_model = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#         results = model(frame_model[frame_cut:])

#         track_bbs_ids = mot_tracker.update(results.xyxy[0][:,:4].cpu())
#         for x1, y1, x2, y2, obj_id in track_bbs_ids:
#             cx1 = int((x1 + x2) / 2)
#             cy1 = int((y1 + y2) / 2)
#             width = abs(x2 - x1)
#             height = abs(y2 - y1)
#             diagonal = math.sqrt(width**2 + height**2)
#             print(diagonal,obj_id,frame_count)
#             if diagonal > 60 :
#                 try:
#                     car_image = frame_model[int(y1):int(y2), int(x1):int(x2)]
#                     transform = transforms.Compose([transforms.ToTensor(),
#                                                     transforms.Resize((32,32)),])
#                     car_image = transform(car_image)
#                     car_image = car_image.cuda()
#                 except:
#                     print('no object detected')
#                     continue
#                 with torch.no_grad():
#                     color_output = color_classifier(car_image.unsqueeze(0))
#                     color_prediction = torch.argmax(color_output).item()
#                     color_name = ['black','blue','brown','green','grey','orange','pink','purple','red','white','yellow']
#                     color_class_name = color_name[color_prediction]

#                 try:
#                     body_car_image = frame_model[int(y1):int(y2), int(x1):int(x2)]
#                     transform = transforms.Compose([transforms.ToTensor(),
#                                                     transforms.Resize((256,256)),])
#                     body_car_image = transform(body_car_image)
#                     body_car_image = body_car_image.cuda()
#                 except:
#                     print('no object detected')
#                     continue

#                 with torch.no_grad():
#                     body_output = body_classifier(body_car_image.unsqueeze(0))
#                     body_prediction = torch.argmax(body_output).item()
#                     body_name = ['Heavy-Duty', 'Lorry', 'Luxury', 'Pickup', 'SUV', 'Sedan', 'Van']
#                     body_class_name = body_name[body_prediction]
#                     clf_state = True

            
#                 if obj_id not in object_dict:
#                     object_dict[obj_id] = {
#                         'bboxes': [(frame_count, x1, y1, x2, y2)],
#                         'last_seen_frame': 0,
#                         'color_classifier_preds': [color_prediction],
#                         'body_classifier_preds': [body_prediction]
#                     }
#                 else:
#                     object_dict[obj_id]['bboxes'].append((frame_count, x1, y1, x2, y2))
#                     object_dict[obj_id]['color_classifier_preds'].append(color_prediction)
#                     object_dict[obj_id]['body_classifier_preds'].append(body_prediction)

#                 object_dict[obj_id]['last_seen_frame'] = frame_count

#                 # Calculate the mode prediction of the classifier for the tracked object
#                 color_mode_pred = statistics.mode(object_dict[obj_id]['color_classifier_preds'])
#                 object_dict[obj_id]['color_mode_pred'] = color_mode_pred

#                 body_mode_pred = statistics.mode(object_dict[obj_id]['body_classifier_preds'])
#                 object_dict[obj_id]['body_mode_pred'] = body_mode_pred

#             cv2.putText(frame, str(obj_id), (cx1, cy1), 0, 0.5, (255, 255, 255), 2)
#             if clf_state == True:
#                 cv2.putText(frame, color_name[color_mode_pred], (int(x1), int(y1)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (203, 192, 255), 2)
#                 cv2.putText(frame,  body_name[body_mode_pred], (int(cx1), int(y2)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (203, 192, 255), 2)

#         output_file.stdin.write(frame.tobytes())

#         for obj_id, obj_data in list(object_dict.items()):
#             if frame_count - obj_data['last_seen_frame'] > mot_tracker.max_age:
#                 object_dict.pop(obj_id)

#         frame_count += 1
    
#     cap.release()
#     output_file.stdin.close()
#     output_file.wait()
    
#     with open("object_tracks.json", "w") as f:
#         json.dump(object_dict, f, indent=4)

In [15]:
if __name__ == '__main__':
    video_path = 'traffic-27260.mp4'
    output_path = 'track_test.mp4'
    cap = cv2.VideoCapture(video_path)

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    ffmpeg_cmd = f"ffmpeg -y -f rawvideo -pix_fmt bgr24 -s {frame_width}x{frame_height} -r {fps} -i - -c:v libx264 -preset fast -crf 30 -pix_fmt nv12 -an -vcodec libx264 {output_path}"

    output_file = subprocess.Popen(ffmpeg_cmd.split(' '), stdin=subprocess.PIPE)
    
    mot_tracker = Sort(max_age=30, min_hits=60) 

    object_dict = {}

    frame_cut = 0
    frame_count = 0
    clf_state = False
    while True:
        ret, frame = cap.read()
        if not ret:
            print('Video processing completed')
            break

        frame_model = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = model(frame_model[frame_cut:])

        track_bbs_ids = mot_tracker.update(results.xyxy[0][:,:4].cpu())
        
        color_thread = threading.Thread(target=color_classifier)
        color_thread.start()
        
        body_thread = threading.Thread(target=body_classifier)
        body_thread.start()
        for x1, y1, x2, y2, obj_id in track_bbs_ids:
            cx1 = int((x1 + x2) / 2)
            cy1 = int((y1 + y2) / 2)
            width = abs(x2 - x1)
            height = abs(y2 - y1)
            diagonal = math.sqrt(width**2 + height**2)
            
            if diagonal > 60 :
                try:
                    car_image = frame_model[int(y1):int(y2), int(x1):int(x2)]
                    transform = transforms.Compose([transforms.ToTensor(),
                                                    transforms.Resize((32,32)),])
                    car_image = transform(car_image)
                    car_image = car_image.cuda()
                except:
                    print('no object detected')
                    continue
                with torch.no_grad():
                    color_output = color_classifier(car_image.unsqueeze(0))
                    color_prediction = torch.argmax(color_output).item()
                    color_name = ['black','blue','brown','green','grey','orange','pink','purple','red','white','yellow']
                    color_class_name = color_name[color_prediction]

                try:
                    body_car_image = frame_model[int(y1):int(y2), int(x1):int(x2)]
                    transform = transforms.Compose([transforms.ToTensor(),
                                                    transforms.Resize((256,256)),])
                    body_car_image = transform(body_car_image)
                    body_car_image = body_car_image.cuda()
                except:
                    print('no object detected')
                    continue

                with torch.no_grad():
                    
                    body_output = body_classifier(body_car_image.unsqueeze(0))
                    body_prediction = torch.argmax(body_output).item()
                    body_name = ['Heavy-Duty', 'Lorry', 'Luxury', 'Pickup', 'SUV', 'Sedan', 'Van']
                    body_class_name = body_name[body_prediction]
                    clf_state = True

            
                if obj_id not in object_dict:
                    object_dict[obj_id] = {
                        'bboxes': [(frame_count, x1, y1, x2, y2)],
                        'last_seen_frame': 0,
                        'color_classifier_preds': [color_prediction],
                        'body_classifier_preds': [body_prediction]
                    }
                else:
                    object_dict[obj_id]['bboxes'].append((frame_count, x1, y1, x2, y2))
                    object_dict[obj_id]['color_classifier_preds'].append(color_prediction)
                    object_dict[obj_id]['body_classifier_preds'].append(body_prediction)

                object_dict[obj_id]['last_seen_frame'] = frame_count

                # Calculate the mode prediction of the classifier for the tracked object
                color_mode_pred = statistics.mode(object_dict[obj_id]['color_classifier_preds'])
                object_dict[obj_id]['color_mode_pred'] = color_mode_pred

                body_mode_pred = statistics.mode(object_dict[obj_id]['body_classifier_preds'])
                object_dict[obj_id]['body_mode_pred'] = body_mode_pred

            cv2.putText(frame, str(obj_id), (cx1, cy1), 0, 0.5, (255, 255, 255), 2)
            if clf_state == True:
                cv2.putText(frame, color_name[color_mode_pred], (int(x1), int(y1)), cv2.FONT_HERSHEY_SIMPLEX, 1, (203, 192, 255), 2)
                cv2.putText(frame,  body_name[body_mode_pred], (int(cx1), int(y2)), cv2.FONT_HERSHEY_SIMPLEX, 1, (203, 192, 255), 2)

        output_file.stdin.write(frame.tobytes())

        for obj_id, obj_data in list(object_dict.items()):
            if frame_count - obj_data['last_seen_frame'] > mot_tracker.max_age:
                object_dict.pop(obj_id)

        frame_count += 1
    color_thread.join()
    body_thread.join()
    cap.release()
    output_file.stdin.close()
    output_file.wait()
    
    with open("object_tracks.json", "w") as f:
        json.dump(object_dict, f, indent=4)
        

Exception in thread Thread-2776:
Traceback (most recent call last):
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
Exception in thread Thread-2777:
Traceback (most recent call last):
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 917, in run
    self.run()
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ahmed\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ahmed\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
TypeError: forward() missing 1 required positional argument: 'x'
    return forward_call(*args, **kwargs)
TypeError: forward() missing 1 required positional argument: 'x'
Exception in threa

    return forward_call(*args, **kwargs)
TypeError: forward() missing 1 required positional argument: 'x'
Exception in thread Thread-2794:
Traceback (most recent call last):
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
Exception in thread Thread-2795:
Traceback (most recent call last):
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 917, in run
    self.run()
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ahmed\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ahmed\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
TypeError: forward() missing 1 required positional argument: 'x'
    return forward

    return forward_call(*args, **kwargs)
TypeError: forward() missing 1 required positional argument: 'x'
Exception in thread Thread-2812:
Traceback (most recent call last):
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
Exception in thread Thread-2813:
Traceback (most recent call last):
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 917, in run
    self.run()
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ahmed\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ahmed\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
TypeError: forward() missing 1 required positional argument: 'x'
    return forward

    return forward_call(*args, **kwargs)
TypeError: forward() missing 1 required positional argument: 'x'
Exception in thread Thread-2830:
Traceback (most recent call last):
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
Exception in thread Thread-2831:
Traceback (most recent call last):
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 917, in run
    self.run()
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ahmed\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ahmed\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
TypeError: forward() missing 1 required positional argument: 'x'
    return forward

    return forward_call(*args, **kwargs)
TypeError: forward() missing 1 required positional argument: 'x'
Exception in thread Thread-2848:
Traceback (most recent call last):
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
Exception in thread Thread-2849:
Traceback (most recent call last):
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 917, in run
    self.run()
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ahmed\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ahmed\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
TypeError: forward() missing 1 required positional argument: 'x'
    return forward

...

no object detected
no object detected


Exception in thread Thread-3438:
Traceback (most recent call last):
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
Exception in thread Thread-3439:
Traceback (most recent call last):
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 917, in run
    self.run()
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ahmed\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ahmed\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
TypeError: forward() missing 1 required positional argument: 'x'
    return forward_call(*args, **kwargs)
TypeError: forward() missing 1 required positional argument: 'x'
Exception in threa

no object detected
no object detected


Exception in thread Thread-3442:
Traceback (most recent call last):
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
Exception in thread Thread-3443:
Traceback (most recent call last):
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 917, in run
    self.run()
  File "C:\Users\ahmed\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ahmed\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ahmed\anaconda3\lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
TypeError: forward() missing 1 required positional argument: 'x'
    return forward_call(*args, **kwargs)
TypeError: forward() missing 1 required positional argument: 'x'
Exception in threa

no object detected
no object detected
Video processing completed


In [ ]:
results.xyxy[0][:,:4]

In [ ]:
for x1,y1,x2,y2, obj_id in track_bbs_ids:
    print(x1,y1,x2,y2, obj_id)

In [ ]:
import torchvision.transforms as T
from PIL import Image
transform = T.ToPILImage()
img = transform(car_image)
img.show()
output=classifier(car_image.unsqueeze(0))
prediction = torch.argmax(output).item()
print(prediction)